<a href="https://colab.research.google.com/github/simran-bhella/Quora-Question-Pairs/blob/main/GPT2_quora_deduplicationipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install pandas
!pip install torch
!pip install scikit-learn
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch

from transformers import GPT2Tokenizer, GPT2Model, GPT2ForSequenceClassification, GPT2Config, AdamW
from torch.utils.data import DataLoader, Dataset

In [ ]:
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
import torch
import pandas as pd

def preprocess_data(file_path):
    data = pd.read_csv(file_path, sep='\t')

    # Sample 10% of the data
    data_sampled = data.sample(frac=0.2, random_state=42)

    data_sampled = data_sampled.dropna(subset=['question1', 'question2'])
    data_sampled['question1'] = data_sampled['question1'].str.lower().str.replace('[^\w\s]', '', regex=True)
    data_sampled['question2'] = data_sampled['question2'].str.lower().str.replace('[^\w\s]', '', regex=True)
    return data_sampled

# Load and preprocess the dataset
data = preprocess_data('train.csv')

# Split the data into training and testing+validation sets (60% train, 40% test+val)
#train_data is 15% of the original set
train_data, test_val_data = train_test_split(data, test_size=0.4, random_state=42)

# Further split the testing/validation set into testing and validation sets (50% test, 50% val of the 10%, each 5%)
test_data, val_data = train_test_split(test_val_data, test_size=0.5, random_state=42)
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate
2301,300061,422817,422818,how do i approach a girl and start a conversat...,how can you approach a strange girl in india a...,0.0
1311,220492,3857,327594,how do you delete a question you asked on quora,how might my quora question no one likes be de...,1.0
12821,394441,324085,128986,what are some of the best economic books,what is the best book for economics,1.0
9012,18245,34574,34575,how to increase iq level,how can i raise my iq level,1.0
20306,40785,73745,73746,what are the best laptops in india under rs55000,which are the best laptops as of november 2016...,1.0


In [ ]:
configuration = GPT2Config()
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)
model.config.pad_token_id = model.config.eos_token_id

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Custom dataset class
class QuoraDataset(Dataset):
    def __init__(self, tokenizer, dataframe):
        self.tokenizer = tokenizer
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question1 = self.data.iloc[idx, 3]  # Adjust the index for 'question1'
        question2 = self.data.iloc[idx, 4]  # Adjust the index for 'question2'
        combined_question = question1 + " [SEP] " + question2
        labels = self.data.iloc[idx, 5]  # Adjust the index for 'is_duplicate'
        inputs = self.tokenizer.encode_plus(
            combined_question,
            None,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        return {
            'ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

# Create dataset
train_dataset = QuoraDataset(tokenizer, train_data)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

for epoch in range(3):  # Number of epochs
    model.train()
    total_loss = 0

    print(f"{epoch=}")
    for batch in train_loader:
        optimizer.zero_grad()



        inputs = batch['ids'].to(device)
        masks = batch['mask'].to(device)
        labels = batch['labels'].to(device)

        print(". ", end="")
        outputs = model(inputs, attention_mask=masks, labels=labels)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        print("- ", end="")


    print(f"Epoch {epoch+1} Loss {total_loss/len(train_loader)}")

epoch=0
. - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - . - 

In [ ]:
len(train_loader)

NameError: ignored

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def evaluate_model(model, val_loader, device):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            inputs = batch['ids'].to(device)
            masks = batch['mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(inputs, attention_mask=masks)
            logits = outputs.logits

            logits = logits.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()

            predictions.extend(np.argmax(logits, axis=1).flatten())
            true_labels.extend(label_ids.flatten())

    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')
    return accuracy, precision, recall, f1

# Evaluate the model
dev_dataset = QuoraDataset(tokenizer, dev_data)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)

accuracy, precision, recall, f1 = evaluate_model(model, dev_loader, device)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")